In [15]:

def class_funct(value):
    if value > 0:
        return 1
    if value < 0:
        return 0
    else:
        return 0
if __name__ == "__main__":
    start = 100
    for value in range(1400,1420, 10):
        """Basic data loading and removing unnessicary features for training data"""
        df = pd.read_csv("Updated_Model_Daily.csv")
        df = df.iloc[:,1:]
        df.dropna(inplace = True)
        df.reset_index(inplace = True, drop = True)
        training_df = df.drop(["Date","High","Low","Close", "Close/Last", "Open", "Volume"], axis = 1)

        training_df["Class_Target"] = training_df["Target"].apply(class_funct)

        """Setting up training/test split (Test data is last month of data and is untouched during training to
        simulate unknown future market conditions"""
        X_train = training_df.drop(["Target","Class_Target"],axis =1)[0:value]
        y_train = training_df["Class_Target"][0:value]

        X_test = training_df.drop(["Target","Class_Target"],axis =1)[value+2:value+7]
        y_test = training_df["Class_Target"][value+2:value+7]
        column_list = X_train.columns
        X_train, X_valid, y_train, y_valid = train_test_split( X_train, y_train, test_size = .10)
        scaler = SS().fit(X_train)
        X_test = scaler.transform(X_test)
        X_train = scaler.transform(X_train)
        X_valid = scaler.transform(X_valid)
        X_test = pd.DataFrame(X_test, columns = column_list)
        X_train = pd.DataFrame(X_train, columns = column_list)
        X_valid = pd.DataFrame(X_valid, columns = column_list)

        # model = keras.models.load_model("NN")
        # prediction = model.predict(X_test)
        # up_pred = []
        # for val in prediction:
        #     if val < .5:
        #         up_pred.append(0)
        #     else:
        #         up_pred.append(1)
        """Defining and training Model, hyperparamter tuning was done in Jupyter, random grid search was used 
        valiadation data created from training data"""
        depth = 15
        n_estimaters = 500
        split = 10
        model_LR =  LR()#RandomForestClassifier(n_estimators = 500, max_depth = 13, min_samples_split = 8) #GradientBoostingClassifier(n_estimators = 5000, learning_rate = .009) 
        model_LR.fit(X_train,y_train)
        model_class =  RandomForestClassifier(n_estimators = n_estimaters, max_depth = depth, min_samples_split = split, class_weight = {0:1, 1:1.05}) #GradientBoostingClassifier(n_estimators = 5000, learning_rate = .009) 
        model_class.fit(X_train,y_train)
        while (accuracy_score(y_valid,model_class.predict(X_valid))) < .50:
                #print("Hit")
                if depth <= 0 or n_estimaters <= 0:
                    depth = 15
                    n_estimaters = 500
                model_class =  RandomForestClassifier(n_estimators = n_estimaters, max_depth = depth, min_samples_split = split, class_weight = {0:1, 1:1.05}, n_jobs= 2) #GradientBoostingClassifier(n_estimators = 5000, learning_rate = .009) 
                model_class.fit(X_train,y_train)
                depth = random.randint(5,20)
                n_estimaters = random.randint(400,600)
                split = random.randint(5,20)
        #print("Out")
        model_boost =  KNeighborsClassifier(n_neighbors = 2)#GradientBoostingClassifier(n_estimators = 5000, learning_rate = .009) 
        model_boost.fit(X_train,y_train)
        #print(accuracy_score(y_test,model_boost.predict(X_test)))
        
        arr = confusion_matrix(y_train,(model_LR.predict(X_train))).ravel()
        arr2 = confusion_matrix(y_train,(model_class.predict(X_train))).ravel()
        arr3 = confusion_matrix(y_train,(model_boost.predict(X_train))).ravel()
        fp = round((arr2[1]+arr3[1])/2)
        fn = round((arr2[2]+arr3[2])/2)
        """Predictions are made off a majority vote by models, this is for in traning data"""
        prediction1 = model_class.predict(X_train)
        prediction2 = model_LR.predict(X_train)
        prediction3 = model_boost.predict(X_train)
        prediction = []
        for index in range(0,len(X_train)):
            if prediction1[index] == 1:
                if  prediction3[index] == 1:
                    prediction.append(1)
                else:
                    if fp > fn:
                        prediction.append(0)
                    else:
                        prediction.append(1)
            else:
                if prediction3[index] == 0:
                    prediction.append(0)
                else:
                    if fp > fn:
                        prediction.append(0)
                    else:
                        prediction.append(1)
        
        print(accuracy_score(y_train,prediction))
        test_df = X_train.copy()
        test_df["Prediction"] = prediction
        test_df["Target"] = y_train
        new_target = []
        for index in test_df.index:
            if test_df.iloc[index]["Prediction"] == test_df.iloc[index]["Prediction"]:
                new_target.append(0)
            else:
                new_target.append(1)
        print(new_target[1:100])      
        model_residuals = RandomForestClassifier(n_estimators = n_estimaters, max_depth = depth, min_samples_split = split)
        model_residuals.fit(X_train,new_target)
        print(accuracy_score(new_target, model_residuals.predict(X_train)))
        print(confusion_matrix(new_target,(model_residuals.predict(X_train))).ravel())
        


        """Predictions are made off a majority vote by models, probabity conditial is also defined"""
        prediction1 = model_class.predict(X_test)
        prediction2 = model_LR.predict(X_test)
        prediction3 = model_boost.predict(X_test)
        prediction = []
        for index in range(0,5):
            if prediction1[index] == 1:
                if  prediction3[index] == 1:
                    prediction.append(1)
                else:
                    if fp > fn:
                        prediction.append(0)
                    else:
                        prediction.append(1)
            else:
                if prediction3[index] == 0:
                    prediction.append(0)
                else:
                    if fp > fn:
                        prediction.append(0)
                    else:
                        prediction.append(1)
                        
            
        """Test df is created for market simulation, Open and target columns are brought back in
        total compouned equity is split over 3 trading days equally"""
        test = X_test
        test["Prediction"] = prediction
        target = list(df["Target"][value+2:value+7])
        test["Target"] = target

        test.reset_index(inplace = True, drop = True)

        """Simulating model deployment, with $30 investments every day based on prediction and gain/loss
        determined by actual Target."""
        total_gain_loss = []
        for index in test.index:
            """Code Block for purchase positions"""
            if test.iloc[index]["Prediction"] == 1:
                """Conditional based off high false positives rates from training data"""
                if False:#random.randint(0,(avg1+avg2)) < avg1:
                    value = (-start/5*test.iloc[index]["Target"])+start/5
                    total_gain_loss.append(value)
                else:
                    value = (start/5*test.iloc[index]["Target"])+start/5
                    total_gain_loss.append(value)
                """Code block for shorting positions""" 
            elif test.iloc[index]["Prediction"] == 0:
                value = (-start/5*test.iloc[index]["Target"])+start/5
                total_gain_loss.append(value)
            else:
                total_gain_loss.append(start/5)

        start = (sum(total_gain_loss))
    print(start)


0.9904761904761905
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1.0
[1260]
0.9881796690307328
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1.0
[1269]
103.7027548221956


In [4]:

def class_funct(value):
    if value > 0:
        return 1
    if value < 0:
        return 0
    else:
        return 0

In [1]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression as LR
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler as SS
import pandas as pd
import random
import numpy as np
from sklearn.cluster import KMeans
#import keras
from sklearn.model_selection import train_test_split

In [116]:
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
import scipy
from functools import *
import random

In [126]:
metric_funct = partial(scipy.spatial.distance.euclidean, w = arr)

In [124]:
arr = [1]*19
for index in range(0,19):
    value = (random.randint(0,10))/10
    arr[index] = arr[index]*value
    
    

In [217]:
final = []
final_2 = []
#final.append(arr)

In [450]:
print(dic[3])

{1: 'High_Risk_Ideal'}


In [458]:
final[3]

[0.3,
 0.9,
 0.2,
 1.0,
 0.0,
 0.6,
 0.8,
 0.3,
 0.4,
 0.2,
 0.5,
 0.6,
 0.4,
 0.1,
 0.8,
 0.9,
 1.0,
 0.5,
 1.0]

In [680]:
#final = []
for final_index in range(6,7):
    arr = final[final_index]
    #print(dic[57])
    """Creation of randomly set weight array"""
#     arr = [1]*19
#     for index in range(0,19):
#         value = (random.randint(0,10))/10
#         arr[index] = arr[index]*value
    metric_funt = partial(scipy.spatial.distance.euclidean, w = arr)
    value = 1300
    
    
    """Basic data loading and removing unnessicary features for training data"""
    df = pd.read_csv("Updated_Model_Daily.csv")
    df = df.iloc[:,1:]
    df.dropna(inplace = True)
    df.reset_index(inplace = True, drop = True)
    training_df = df.drop(["Date","High","Low","Close", "Close/Last", "Open", "Volume"], axis = 1)

    training_df["Class_Target"] = training_df["Target"].apply(class_funct)

    """Setting up training/test split (Test data is last month of data and is untouched during training to
    simulate unknown future market conditions"""
    X_train = training_df.drop(["Target","Class_Target"],axis =1)[0:value]
    y_train = training_df["Class_Target"][0:value]

    X_test = training_df.drop(["Target","Class_Target"],axis =1)[value:]
    y_test = training_df["Class_Target"][value:]
    column_list = X_train.columns
    #X_train, X_valid, y_train, y_valid = train_test_split( X_train, y_train, test_size = .10)
    scaler = SS().fit(X_train)
    X_test = scaler.transform(X_test)
    X_train = scaler.transform(X_train)
    #X_valid = scaler.transform(X_valid)
    X_test = pd.DataFrame(X_test, columns = column_list)
    X_train = pd.DataFrame(X_train, columns = column_list)
    #X_valid = pd.DataFrame(X_valid, columns = column_list)
    cluster = OPTICS(metric = metric_funt)#DBSCAN(eps = 1, metric = metric_funct, min_samples = 10) 
    cluster.fit(X_train)
    X_train["Cluster"] = cluster.labels_
    X_train["Target"] = training_df["Target"][0:1300]
    count = 0
    cluster_id = {}
    print("Index: ", final_index, " sum of clusters ",len(X_train[X_train["Cluster"] >= 0]))
    for val in range(-1,len(X_train["Cluster"].unique()-1)):
        temp = X_train[X_train["Cluster"]== val ]
        if len(temp) > highest and val != -1:
            highest = len(temp)
            high_val = val
        pos = (temp[temp["Target"] > 0])["Target"].mean()
        neg = (temp[temp["Target"] < 0])["Target"].mean()
        print("Average volitilaty value: ", temp.drop("Cluster", axis = 1).var().mean())
        #print(val, " Average pos value: ", pos)
        #print(val, " Average neg value: ", neg) 
        if (pos/neg < -1.5) or (neg/pos < -1.5):
            count = count+ 1
            cluster_id[val] = "Mid_Risk"
        if (pos/neg < -2) or (neg/pos < -2):
            count = count+ 1
            cluster_id[val] = "High_Risk"
    if count >= 2:
        pass
        #final.append([arr, cluster_id])


Index:  0  sum of clusters  243
Average volitilaty value:  0.8749304431809269
Average volitilaty value:  0.01472332540448749
Average volitilaty value:  0.022368606283153403
Average volitilaty value:  0.01270751105821632
Average volitilaty value:  0.023003451451518893
Average volitilaty value:  0.026830511027615328
Average volitilaty value:  0.04837991130479557
Average volitilaty value:  0.02758890452612046
Average volitilaty value:  0.019882846303373856
Average volitilaty value:  0.019812210423191576
Average volitilaty value:  0.0374592487971256
Average volitilaty value:  0.026015698838901015
Average volitilaty value:  0.04563169830397183
Average volitilaty value:  0.03161544496103594
Average volitilaty value:  0.03616526963376822
Average volitilaty value:  0.024325366132827292
Average volitilaty value:  0.05693275108771373
Average volitilaty value:  0.016236163922457126
Average volitilaty value:  0.03207893042133485
Average volitilaty value:  0.048848830512417445
Average volitilaty va

In [683]:
index_6 = 303
index_13 = 309
index_28 = 306
index_48 = 362
index_66 = 327
(X_train[X_train["Cluster"] == 2])

,30_Day_Preformance,14_Day_Preformance,7_Day_Preformance,30_Day_Preformance_mean,14_Day_Preformance_mean,7_Day_Preformance_mean,30_Day_Vmean,14_Day_Vmean,7_Day_Vmean,After_Hours,...,14_Day_Preformance_SP500,7_Day_Preformance_SP500,30_Day_Preformance_mean_SP500,14_Day_Preformance_mean_SP500,7_Day_Preformance_mean_SP500,30_Day_Preformance_tech,14_Day_Preformance_tech,7_Day_Preformance_tech,Cluster,Target
15,1.038443,0.029921,0.045143,0.447837,-0.026138,-0.175779,-0.095978,-0.733737,-0.587861,0.193013,...,-0.056658,-0.183142,0.178272,-0.223238,-0.434923,0.184553,-0.014098,-0.120181,2,0.003316
16,0.964268,0.150585,-0.002940,0.431482,0.025752,-0.109898,-0.135903,-0.736818,-0.605682,-0.284732,...,0.111332,-0.259154,0.261044,-0.083502,-0.212131,0.288009,0.134712,-0.063992,2,-0.001867
17,0.889763,-0.020738,-0.389804,0.304476,-0.099730,-0.253289,-0.162037,-0.755685,-0.830805,0.210086,...,-0.027179,-0.493638,0.217015,-0.136834,-0.209158,0.333951,0.018548,-0.115140,2,0.011595
18,1.036230,0.006175,-0.133826,0.288386,-0.052858,-0.105084,-0.175397,-0.809989,-0.953161,-0.013791,...,-0.125442,-0.331388,0.147638,-0.209713,-0.202090,0.394287,0.061211,0.204429,2,0.009978
19,1.024744,-0.087198,-0.207134,0.223062,-0.070138,-0.096506,-0.233719,-0.850048,-1.037310,-0.197790,...,-0.174369,-0.217995,0.134987,-0.187870,-0.117605,0.437541,-0.010256,0.250935,2,0.004160


In [581]:
model = KNeighborsClassifier(n_neighbors = 1, p = 1)
model.fit(X_train.drop(["Target", "Cluster"], axis = 1), X_train["Cluster"])

KNeighborsClassifier(n_neighbors=1, p=1)

In [582]:
X_test["Cluster"] = model.predict(X_test)

In [583]:
X_test["Cluster"].unique()

array([-1,  3, 25, 24,  7,  8,  9,  0])

In [585]:
X_test["Target"] = list(training_df["Target"][1300:])

In [681]:
X_test[(X_test["Cluster"] == 2) & (X_test["Target"] < 0)].mean()


KeyError: 'Cluster'

In [466]:
final = []
dic = []
"""Loop for 100 random arrays"""
for _ in range(0,500):
    """Creation of randomly set weight array"""
    arr = [1]*19
    for index in range(0,19):
        value = (random.randint(0,100))/100
        arr[index] = arr[index]*value
    metric_funct = partial(scipy.spatial.distance.euclidean, w = arr)
    #print(arr)
    
    """Basic data loading and removing unnessicary features for training data"""
    df = pd.read_csv("Updated_Model_Daily.csv")
    df = df.iloc[:,1:]
    df.dropna(inplace = True)
    df.reset_index(inplace = True, drop = True)
    training_df = df.drop(["Date","High","Low","Close", "Close/Last", "Open", "Volume"], axis = 1)

    """Setting up training data for clustering, only 1300 samples are used as the predictive model will
    used for the 140 days after (index values 1300:1440) """
    X_train = training_df.drop(["Target"],axis =1)[0:1300]
    #y_train = training_df["Class_Target"][0:1300]
    column_list = X_train.columns
    scaler = SS().fit(X_train)
    X_train = scaler.transform(X_train)
    X_train = pd.DataFrame(X_train, columns = column_list)


    """For each new set of weights a new DBSCAN model is trained and the clusters are tested and if the
    diffrence between postive targets and negative targets are greater than 1.5 the weights are saved to
    an array and tagged"""
    cluster = DBSCAN(eps = 1, metric = metric_funct, min_samples = 10) 
    cluster.fit(X_train)
    X_train["Cluster"] = cluster.labels_
    X_train["Target"] = training_df["Target"][0:1300]
    count = 0
    cluster_id = {}
    for val in range(-1,len(X_train["Cluster"].unique())-1):
        temp = X_train[X_train["Cluster"]== val ]
        #print(temp)
        pos = (temp[temp["Target"] > 0])["Target"].mean()
        neg = (temp[temp["Target"] < 0])["Target"].mean()
        #print(val, " Average pos value: ", pos)
        #print(val, " Average neg value: ", neg)
        if len(temp) < 20:
            if (pos/neg < -1.5) or (neg/pos < -1.5):
                count = count+ 1
                cluster_id[val] = "Mid_Risk"
            if (pos/neg < -2) or (neg/pos < -2):
                count = count+ 1
                cluster_id[val] = "High_Risk"
        else:
            if (pos/neg < -1.5) or (neg/pos < -1.5):
                count = count+ 1
                cluster_id[val] = "Mid_Risk_Ideal"
            if (pos/neg < -2) or (neg/pos < -2):
                count = count+ 1
                cluster_id[val] = "High_Risk_Ideal"
    if count >= 2:
        final.append(arr)
        dic.append(cluster_id)

In [467]:
index_arr = []
for index in range(0,len(final)):
    temp = dic[index]
    if "High_Risk_Ideal" in temp.values() or ("Mid_Risk_Ideal" in temp.values()):
        index_arr.append(index)
        
index_arr

[17, 31, 42, 52, 57, 68, 75, 78, 80, 120, 134, 142, 149, 152, 160]

In [427]:
len(final)

6

In [163]:
count = 0
for val in range(-1,len(X_train["Cluster"].unique())-1):
    temp = X_train[X_train["Cluster"]== val ]
    pos = (temp[temp["Target"] > 0])["Target"].mean()
    neg = (temp[temp["Target"] < 0])["Target"].mean()
    if pos/neg < -1.5 or neg/pos < -1.5:
        count += 1
    print(val, " Average pos value: ", pos)
    print(val, " Average neg value: ", neg)


-1  Average pos value:  nan
-1  Average neg value:  -0.0122164048865619
0  Average pos value:  0.021105657160884014
0  Average neg value:  -0.02070589260065875


In [341]:
 = X_train[(X_train["Cluster"]== 1)]

,30_Day_Preformance,14_Day_Preformance,7_Day_Preformance,30_Day_Preformance_mean,14_Day_Preformance_mean,7_Day_Preformance_mean,30_Day_Vmean,14_Day_Vmean,7_Day_Vmean,After_Hours,...,14_Day_Preformance_SP500,7_Day_Preformance_SP500,30_Day_Preformance_mean_SP500,14_Day_Preformance_mean_SP500,7_Day_Preformance_mean_SP500,30_Day_Preformance_tech,14_Day_Preformance_tech,7_Day_Preformance_tech,Cluster,Target
87,-0.964148,-1.052576,-0.246318,-1.072488,-0.971345,-0.157566,0.043559,0.449200,0.912034,-0.237347,...,0.041120,0.056561,0.072211,-0.071301,-0.043909,0.163670,-0.378262,0.210246,1,0.005223
88,-0.934025,-1.066127,-0.416153,-1.015525,-0.841998,-0.092614,-0.024899,0.484710,0.228757,0.098357,...,-0.141334,-0.095305,0.033122,-0.132182,-0.132440,0.121754,-0.358754,-0.079029,1,0.000343
89,-0.903499,-1.197693,-0.557875,-1.058437,-0.857883,-0.194509,-0.050859,0.421685,0.027462,0.317856,...,-0.237149,-0.278186,-0.017430,-0.186020,-0.206699,0.085331,-0.493492,-0.208437,1,-0.009647
91,-1.121732,-1.270764,-0.030936,-1.019171,-0.691832,-0.270965,-0.045565,0.489041,0.000613,0.487319,...,-0.160013,-0.067739,-0.034066,-0.167392,-0.200875,-0.094530,-0.675142,0.154170,1,-0.009172
92,-1.150038,-1.341395,-0.090370,-1.037828,-0.661138,-0.431201,-0.050166,0.516109,-0.341955,-0.378793,...,-0.261813,-0.348553,-0.145150,-0.301583,-0.394025,-0.391267,-1.078078,-0.362790,1,-0.003253
93,-0.999184,-1.314883,-0.528750,-0.972530,-0.495907,-0.407636,-0.042186,0.522744,-0.478110,0.657097,...,-0.142804,-0.265711,0.012587,-0.063877,-0.032503,-0.253506,-1.114971,-0.475345,1,-0.008362
155,-0.851913,-1.070474,-1.057422,-0.953894,-0.891099,-0.502083,0.151077,0.859222,0.755478,-0.300676,...,0.153541,-0.132159,0.144476,0.004502,-0.110842,0.147070,0.133326,-0.416224,1,0.002081
156,-0.919936,-0.953908,-0.968021,-0.921262,-0.786184,-0.288510,0.135667,0.884752,0.718520,0.219693,...,0.214666,-0.185112,0.131754,-0.035355,-0.110907,0.163163,0.127480,-0.457844,1,-0.000520
157,-0.879474,-0.985097,-0.849663,-0.965558,-0.816638,-0.262250,0.113867,0.837823,0.737537,0.080075,...,0.154633,-0.098579,0.196250,0.023937,0.045804,0.309321,0.275897,-0.207690,1,0.008289
158,-0.658894,-1.073538,-0.425547,-0.830092,-0.569775,0.114787,0.109318,0.799114,0.389842,-0.028968,...,0.089945,0.058868,0.311946,0.167003,0.288724,0.682292,0.302035,0.329477,1,-0.005056


In [504]:
import math
from sklearn.cluster import OPTICS

In [339]:
cluster.core_points

AttributeError: 'DBSCAN' object has no attribute 'core_points'

In [527]:
import numpy as np
import scipy as sp

def dbscan_predict(dbscan_model, X_new, metric=sp.spatial.distance.cosine):
    # Result is noise by default
    y_new = np.ones(shape=len(X_new), dtype=int)*-1 

    # Iterate all input samples for a label
    for j, x_new in enumerate(X_new):
        # Find a core sample closer than EPS

        for i, x_core in enumerate(dbscan_model.components_):
            #print(metric(x_new, x_core))
            if metric(x_new, x_core) < dbscan_model.eps:
                
                # Assign label of x_core to x_new
                y_new[j] = dbscan_model.labels_[dbscan_model.core_sample_indices_[i]]
                break

    return y_new
X_test_np = X_test.to_numpy()
tag = dbscan_predict(cluster,X_test_np , metric = metric_funct)
X_test["Cluster"] = tag
X_test["Target"] = list(training_df["Target"][1300:])

AttributeError: 'OPTICS' object has no attribute 'components_'

In [495]:
(X_test[(X_test["Cluster"] == 0) & (X_test["Target"] < 0)]).mean()
(X_test[(X_test["Cluster"] == 1)])

,30_Day_Preformance,14_Day_Preformance,7_Day_Preformance,30_Day_Preformance_mean,14_Day_Preformance_mean,7_Day_Preformance_mean,30_Day_Vmean,14_Day_Vmean,7_Day_Vmean,After_Hours,...,14_Day_Preformance_SP500,7_Day_Preformance_SP500,30_Day_Preformance_mean_SP500,14_Day_Preformance_mean_SP500,7_Day_Preformance_mean_SP500,30_Day_Preformance_tech,14_Day_Preformance_tech,7_Day_Preformance_tech,Cluster,Target


In [409]:
X_test.drop(["Cluster", "Target"], axis = 1, inplace = True)

In [485]:
sp.spatial.distance.cosine(final[42], final[31])

0.3178701360027262

In [29]:
arr = [1]*(1390)
value_for = 0
for index in range(1390-1,-1,-1):
    arr[index] = arr[index]-value_for
    value_for += .00015

In [594]:
for value in (0,1):
    print(value)

0
1


In [606]:
import requests
from bs4 import BeautifulSoup

In [622]:
URL = "https://finance.yahoo.com/quote/AAPL/options?p=AAPL"
headers = {'User-Agent':'Mozilla/5.0'}
page = requests.get(URL, headers = headers)

In [623]:
soup = BeautifulSoup(page.content, "html.parser")

In [627]:
soup

<!DOCTYPE html>
<html class="NoJs desktop" data-color-theme="light" id="atomic" lang="en-US"><head prefix="og: https://ogp.me/ns#"><script>window.performance && window.performance.mark && window.performance.mark('PageStart');</script><meta charset="utf-8"/><title>Apple Inc. (AAPL) Options Chain - Yahoo Finance</title><meta content="AAPL, Apple Inc., AAPL options, Apple Inc. options, AAPL option chain, Apple Inc. option chain, option, option chain, AAPL stock chart, Apple Inc. stock chart, stock chart, stocks, quotes, finance" name="keywords"/><meta content="on" http-equiv="x-dns-prefetch-control"/><meta content="on" property="twitter:dnt"/><meta content="458584288257241" property="fb:app_id"/><meta content="#400090" name="theme-color"/><meta content="width=device-width, initial-scale=1" name="viewport"/><meta content="View the basic AAPL option chain and compare options of Apple Inc. on Yahoo Finance." lang="en-US" name="description"/><meta content="guce.yahoo.com" name="oath:guce:cons

In [652]:
for val in range(0,100):
    mydivs = soup.find_all("tr", {"class": "data-row"+str(val)+" Bgc($hoverBgColor):h BdT Bdc($seperatorColor) H(33px) in-the-money Bgc($hoverBgColor)"})
    if len(mydivs) == 0:
        print("No value: ", val)

No value:  30
No value:  31
No value:  32
No value:  58
No value:  59
No value:  60
No value:  61
No value:  62
No value:  63
No value:  64
No value:  65
No value:  66
No value:  67
No value:  68
No value:  69
No value:  70
No value:  71
No value:  72
No value:  73
No value:  74
No value:  75
No value:  76
No value:  77
No value:  78
No value:  79
No value:  80
No value:  81
No value:  82
No value:  83
No value:  84
No value:  85
No value:  86
No value:  87
No value:  88
No value:  89
No value:  90
No value:  91
No value:  92
No value:  93
No value:  94
No value:  95
No value:  96
No value:  97
No value:  98
No value:  99


In [677]:
training_df

,30_Day_Preformance,14_Day_Preformance,7_Day_Preformance,30_Day_Preformance_mean,14_Day_Preformance_mean,7_Day_Preformance_mean,30_Day_Vmean,14_Day_Vmean,7_Day_Vmean,After_Hours,...,30_Day_Preformance_SP500,14_Day_Preformance_SP500,7_Day_Preformance_SP500,30_Day_Preformance_mean_SP500,14_Day_Preformance_mean_SP500,7_Day_Preformance_mean_SP500,30_Day_Preformance_tech,14_Day_Preformance_tech,7_Day_Preformance_tech,Class_Target
0,0.170294,0.113832,0.049485,0.102727,0.059409,0.029251,1.227294e+08,1.391933e+08,1.098839e+08,-0.003703,...,0.037312,0.016075,0.018001,0.023510,0.017207,0.013172,0.085321,0.017457,0.009089,1
1,0.171083,0.110775,0.039298,0.098074,0.052443,0.023296,1.236374e+08,1.418384e+08,1.149091e+08,-0.001181,...,0.038928,0.022018,0.023420,0.028409,0.022155,0.016669,0.093724,0.028458,0.014104,1
2,0.165459,0.117175,0.028550,0.087645,0.040223,0.013395,1.238318e+08,1.424225e+08,1.059842e+08,0.001812,...,0.032991,0.024934,0.023350,0.024192,0.017670,0.010442,0.090795,0.027118,0.011617,1
3,0.166552,0.124474,0.034789,0.091126,0.040812,0.017781,1.238382e+08,1.400900e+08,1.055394e+08,-0.003758,...,0.036885,0.035571,0.025345,0.028272,0.021033,0.012240,0.097131,0.039720,0.017652,1
4,0.156677,0.073998,0.029971,0.087083,0.033832,0.014317,1.228745e+08,1.334384e+08,1.033484e+08,0.001975,...,0.038494,0.033042,0.021122,0.029724,0.021163,0.011290,0.090167,0.035805,0.021440,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1436,-0.019906,0.055405,0.045945,0.015740,0.032655,0.013200,9.396784e+07,8.355253e+07,8.016565e+07,-0.023481,...,-0.018730,0.044071,0.028395,0.023744,0.031225,0.014710,-0.067632,-0.011228,0.032264,1
1437,0.025752,0.094746,0.070864,0.050423,0.063065,0.040460,9.405062e+07,8.997135e+07,9.488051e+07,0.016598,...,0.008296,0.079533,0.052263,0.044207,0.047894,0.030195,-0.042823,0.050850,0.046021,0
1438,0.010952,0.114641,0.085462,0.062692,0.069399,0.043276,9.459959e+07,9.146488e+07,9.965497e+07,-0.011347,...,0.006854,0.086598,0.066908,0.049270,0.047501,0.027972,-0.035708,0.083849,0.068931,0
1439,-0.053356,0.103378,0.011958,0.020260,0.019087,-0.009618,9.368924e+07,9.217528e+07,9.877364e+07,0.011318,...,-0.004779,0.094459,0.024160,0.035876,0.028244,0.005829,-0.054539,0.112848,0.017139,0


In [ ]:
test = pd.Dataframe(columns = ["Avg_60_Day","Avg_30_Day", "Target" ])
for value in range(1200,1400):
    
    """Basic data loading and removing unnessicary features for training data"""
    df = pd.read_csv("Updated_Model_Daily.csv")
    df = df.iloc[:,1:]
    df.dropna(inplace = True)
    df.reset_index(inplace = True, drop = True)
    training_df = df.drop(["Date","High","Low","Close", "Close/Last", "Open", "Volume"], axis = 1)

    training_df["Class_Target"] = training_df["Target"].apply(class_funct)
    training_df["Squared_14_day"] = training_df["14_Day_Preformance"]**2
    training_df["Squared_30_day"] = training_df["30_Day_Preformance"]**2

    X_train = training_df.drop(["Target","Class_Target"],axis =1)[0:value]
    y_train = training_df["Class_Target"][0:value]

    X_test = training_df.drop(["Target","Class_Target"],axis =1)[value+2:value+7]
    y_test = training_df["Class_Target"][value:]
    column_list = X_train.columns
    #X_train, X_valid, y_train, y_valid = train_test_split( X_train, y_train, test_size = .10)
    scaler = SS().fit(X_train)
    X_test = scaler.transform(X_test)
    X_train = scaler.transform(X_train)
    #X_valid = scaler.transform(X_valid)
    X_test = pd.DataFrame(X_test, columns = column_list)
    X_train = pd.DataFrame(X_train, columns = column_list)
    #X_valid = pd.DataFrame(X_valid, columns = column_list)

In [5]:
import pandas as pd
def column_adjust(string):
    return float(string[1:])

df = pd.read_csv("APPL_Day.csv")
df["Date"] = pd.to_datetime(df["Date"])
df["Open"] = df["Open"].apply(column_adjust)
df["Close/Last"] = df["Close/Last"].apply(column_adjust)
#updated_df = df[(df['Date'] > '2016-01-01')]
#updated_df = updated_df[(updated_df['Date'] <= '2022-11-04')]
df = df.sort_values(by='Date')

df.reset_index(drop = True, inplace = True)

In [6]:
df

,Date,Close/Last,Volume,Open,High,Low
0,2012-11-08,19.2054,1053867498,20.0225,$20.0797,$19.1175
1,2012-11-09,19.5379,929191668,19.3007,$19.8172,$19.0614
2,2012-11-12,19.3868,515205674,19.7911,$19.8036,$19.2375
3,2012-11-13,19.3892,532807307,19.2468,$19.66,$19.1557
4,2012-11-14,19.1743,476103970,19.4822,$19.5518,$19.1493
...,...,...,...,...,...,...
2512,2022-11-01,150.6500,80379350,155.0800,$155.45,$149.13
2513,2022-11-02,145.0300,93604620,148.9450,$152.17,$145
2514,2022-11-03,138.8800,97918520,142.0600,$142.8,$138.75
2515,2022-11-04,138.3800,140814800,142.0900,$142.67,$134.38


In [9]:
def target_creation(dataframe):
    target = []
    for index in dataframe.index:
        if index+1 > max(dataframe.index):
            target.append(None)
        else:
            future = dataframe.iloc[(index+1)]["Close/Last"]
            value = (dataframe.iloc[index]["Close/Last"])
            target.append((future-value)/value)
    
    return target

In [10]:
df["Target"] = target_creation(df)

In [11]:
df

,Date,Close/Last,Volume,Open,High,Low,Target
0,2012-11-08,19.2054,1053867498,20.0225,$20.0797,$19.1175,0.017313
1,2012-11-09,19.5379,929191668,19.3007,$19.8172,$19.0614,-0.007734
2,2012-11-12,19.3868,515205674,19.7911,$19.8036,$19.2375,0.000124
3,2012-11-13,19.3892,532807307,19.2468,$19.66,$19.1557,-0.011083
4,2012-11-14,19.1743,476103970,19.4822,$19.5518,$19.1493,-0.020971
...,...,...,...,...,...,...,...
2512,2022-11-01,150.6500,80379350,155.0800,$155.45,$149.13,-0.037305
2513,2022-11-02,145.0300,93604620,148.9450,$152.17,$145,-0.042405
2514,2022-11-03,138.8800,97918520,142.0600,$142.8,$138.75,-0.003600
2515,2022-11-04,138.3800,140814800,142.0900,$142.67,$134.38,0.003902


In [106]:
training_data = []
for index in df.index:
    temp = []
    if index-60 < min(df.index):
        training_data.append(None)
    else:
        for val in range(index-60,index):
            temp.append(df.iloc[(val)]["standard_scaled_open"])
            temp.append(df.iloc[val]["standard_scaled_closed"])
            #target.append((future-value)/value)
        training_data.append(temp)

In [107]:
df["Training_data"] = training_data

In [40]:
df

,Date,Close/Last,Volume,Open,High,Low,Target,Training_data,standard_scaled_open,standard_scaled_closed
0,2012-11-08,19.2054,1053867498,20.0225,$20.0797,$19.1175,0.017313,None,NaN,NaN
1,2012-11-09,19.5379,929191668,19.3007,$19.8172,$19.0614,-0.007734,None,NaN,NaN
2,2012-11-12,19.3868,515205674,19.7911,$19.8036,$19.2375,0.000124,None,NaN,NaN
3,2012-11-13,19.3892,532807307,19.2468,$19.66,$19.1557,-0.011083,None,NaN,NaN
4,2012-11-14,19.1743,476103970,19.4822,$19.5518,$19.1493,-0.020971,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2512,2022-11-01,150.6500,80379350,155.0800,$155.45,$149.13,-0.037305,"[1.1218575927626508, 1.0163905003904796, 0.966...",0.203960,-0.261362
2513,2022-11-02,145.0300,93604620,148.9450,$152.17,$145,-0.042405,"[0.9668109570037485, 1.0515634062189183, 1.293...",-0.401604,-0.825201
2514,2022-11-03,138.8800,97918520,142.0600,$142.8,$138.75,-0.003600,"[1.293852713112455, 1.4279205583641779, 1.5032...",-1.074517,-1.442171
2515,2022-11-04,138.3800,140814800,142.0900,$142.67,$134.38,0.003902,"[1.5032516556467113, 1.3826856780170484, 1.498...",-1.083804,-1.489042


In [25]:
temp = df.iloc[100]["Training_data"]

In [36]:
def moving_average_standard_scaler(df, column, window=90):
    rolling_mean = df[column].rolling(window=window).mean()
    rolling_std = df[column].rolling(window=window).std()
    df['standard_scaled_closed'] = (df[column] - rolling_mean) / rolling_std
    return df

In [37]:
moving_average_standard_scaler(df, "Close/Last")

,Date,Close/Last,Volume,Open,High,Low,Target,Training_data,standard_scaled_open,standard_scaled_closed
0,2012-11-08,19.2054,1053867498,20.0225,$20.0797,$19.1175,0.017313,None,NaN,NaN
1,2012-11-09,19.5379,929191668,19.3007,$19.8172,$19.0614,-0.007734,None,NaN,NaN
2,2012-11-12,19.3868,515205674,19.7911,$19.8036,$19.2375,0.000124,None,NaN,NaN
3,2012-11-13,19.3892,532807307,19.2468,$19.66,$19.1557,-0.011083,None,NaN,NaN
4,2012-11-14,19.1743,476103970,19.4822,$19.5518,$19.1493,-0.020971,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2512,2022-11-01,150.6500,80379350,155.0800,$155.45,$149.13,-0.037305,"[166.37, 164.87, 164.02, 164.92, 167.68, 169.2...",0.203960,-0.261362
2513,2022-11-02,145.0300,93604620,148.9450,$152.17,$145,-0.042405,"[164.02, 164.92, 167.68, 169.24, 170.06, 168.4...",-0.401604,-0.825201
2514,2022-11-03,138.8800,97918520,142.0600,$142.8,$138.75,-0.003600,"[167.68, 169.24, 170.06, 168.49, 169.82, 172.1...",-1.074517,-1.442171
2515,2022-11-04,138.3800,140814800,142.0900,$142.67,$134.38,0.003902,"[170.06, 168.49, 169.82, 172.1, 171.52, 173.19...",-1.083804,-1.489042


In [45]:
df

,Date,Close/Last,Volume,Open,High,Low,Target,Training_data,standard_scaled_open,standard_scaled_closed
0,2012-11-08,19.2054,1053867498,20.0225,$20.0797,$19.1175,0.017313,None,NaN,NaN
1,2012-11-09,19.5379,929191668,19.3007,$19.8172,$19.0614,-0.007734,None,NaN,NaN
2,2012-11-12,19.3868,515205674,19.7911,$19.8036,$19.2375,0.000124,None,NaN,NaN
3,2012-11-13,19.3892,532807307,19.2468,$19.66,$19.1557,-0.011083,None,NaN,NaN
4,2012-11-14,19.1743,476103970,19.4822,$19.5518,$19.1493,-0.020971,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2512,2022-11-01,150.6500,80379350,155.0800,$155.45,$149.13,-0.037305,"[1.1218575927626508, 1.0163905003904796, 0.966...",0.203960,-0.261362
2513,2022-11-02,145.0300,93604620,148.9450,$152.17,$145,-0.042405,"[0.9668109570037485, 1.0515634062189183, 1.293...",-0.401604,-0.825201
2514,2022-11-03,138.8800,97918520,142.0600,$142.8,$138.75,-0.003600,"[1.293852713112455, 1.4279205583641779, 1.5032...",-1.074517,-1.442171
2515,2022-11-04,138.3800,140814800,142.0900,$142.67,$134.38,0.003902,"[1.5032516556467113, 1.3826856780170484, 1.498...",-1.083804,-1.489042


In [108]:
def non_overlapping_windows(lst):
    if lst is None:
        return
    else:
        windows = []
        for i in range(0, len(lst) - 3, 4):
            window = lst[i:i + 4]
            windows.append(window)
        return windows

In [63]:
len(non_overlapping_windows(df.iloc[200]["Training_data"]))

30

In [109]:
df["Windows_4"] = df["Training_data"].apply(non_overlapping_windows)

In [59]:
df

,Date,Close/Last,Volume,Open,High,Low,Target,Training_data,standard_scaled_open,standard_scaled_closed,Windows_4
0,2012-11-08,19.2054,1053867498,20.0225,$20.0797,$19.1175,0.017313,None,NaN,NaN,None
1,2012-11-09,19.5379,929191668,19.3007,$19.8172,$19.0614,-0.007734,None,NaN,NaN,None
2,2012-11-12,19.3868,515205674,19.7911,$19.8036,$19.2375,0.000124,None,NaN,NaN,None
3,2012-11-13,19.3892,532807307,19.2468,$19.66,$19.1557,-0.011083,None,NaN,NaN,None
4,2012-11-14,19.1743,476103970,19.4822,$19.5518,$19.1493,-0.020971,None,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...
2512,2022-11-01,150.6500,80379350,155.0800,$155.45,$149.13,-0.037305,"[1.1218575927626508, 1.0163905003904796, 0.966...",0.203960,-0.261362,"[[1.1218575927626508, 1.0163905003904796, 0.96..."
2513,2022-11-02,145.0300,93604620,148.9450,$152.17,$145,-0.042405,"[0.9668109570037485, 1.0515634062189183, 1.293...",-0.401604,-0.825201,"[[0.9668109570037485, 1.0515634062189183, 1.29..."
2514,2022-11-03,138.8800,97918520,142.0600,$142.8,$138.75,-0.003600,"[1.293852713112455, 1.4279205583641779, 1.5032...",-1.074517,-1.442171,"[[1.293852713112455, 1.4279205583641779, 1.503..."
2515,2022-11-04,138.3800,140814800,142.0900,$142.67,$134.38,0.003902,"[1.5032516556467113, 1.3826856780170484, 1.498...",-1.083804,-1.489042,"[[1.5032516556467113, 1.3826856780170484, 1.49..."


[None]

In [60]:
import tensorflow as tf

In [62]:
tf.keras.layers.MultiHeadAttention(num_heads= 3, key_dim=4)

In [212]:
from keras import layers
import keras
inputs = layers.Input(shape=(30,4))
multi = tf.keras.layers.MultiHeadAttention(num_heads= 96, key_dim=4)
attn_output = multi(inputs,inputs)
#out1 = (inputs + attn_output)
x = layers.Flatten()(attn_output)
x = layers.Dense(300, activation = "relu")(x)
x = layers.Dense(300, activation = "relu")(x)
output = outputs = layers.Dense(1, activation = "sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=output)


In [213]:
model.summary()

Model: "model_22"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_35 (InputLayer)          [(None, 30, 4)]      0           []                               
                                                                                                  
 multi_head_attention_34 (Multi  (None, 30, 4)       7300        ['input_35[0][0]',               
 HeadAttention)                                                   'input_35[0][0]']               
                                                                                                  
 flatten_22 (Flatten)           (None, 120)          0           ['multi_head_attention_34[0][0]']
                                                                                                  
 dense_27 (Dense)               (None, 300)          36300       ['flatten_22[0][0]']      

In [214]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=keras.optimizers.Adam(.00006),
    metrics=[tf.keras.metrics.BinaryAccuracy()]
)

In [123]:
import numpy as np
train = []
for arr in alt_df["Windows_4"]:
    arr = np.asarray(arr)
    train.append(arr)
train = np.asarray(train)

In [126]:
y_train = np.asarray(alt_df["Target_Classification"])

In [129]:
test_x = train[2268:]
test_y = y_train[2268:]
train = train[:2268]
y_train = y_train[:2268]

In [117]:
alt_df = df.copy()[149:]

In [120]:
alt_df["Target_Classification"] = alt_df["Target"].apply(convert)

In [119]:
def convert(val):
    if val < 0:
        return 0
    else:
        return 1

In [215]:
model.fit(x=train, 
          y=y_train, 
          epochs=12,
          batch_size = 32)


Epoch 1/12
71/71 [==============================] - 8s 87ms/step - loss: 0.6928 - binary_accuracy: 0.5273
Epoch 2/12
71/71 [==============================] - 5s 64ms/step - loss: 0.6919 - binary_accuracy: 0.5300
Epoch 3/12
71/71 [==============================] - 5s 66ms/step - loss: 0.6916 - binary_accuracy: 0.5260
Epoch 4/12
71/71 [==============================] - 5s 65ms/step - loss: 0.6917 - binary_accuracy: 0.5300
Epoch 5/12
71/71 [==============================] - 5s 65ms/step - loss: 0.6917 - binary_accuracy: 0.5300
Epoch 6/12
71/71 [==============================] - 5s 65ms/step - loss: 0.6915 - binary_accuracy: 0.5300
Epoch 7/12
71/71 [==============================] - 5s 65ms/step - loss: 0.6914 - binary_accuracy: 0.5300
Epoch 8/12
71/71 [==============================] - 5s 65ms/step - loss: 0.6917 - binary_accuracy: 0.5300
Epoch 9/12
71/71 [==============================] - 5s 65ms/step - loss: 0.6917 - binary_accuracy: 0.5300
Epoch 10/12
71/71 [===========================

In [216]:
model.test_on_batch(test_x,test_y)

[0.6891952753067017, 0.5099999904632568]